In [1]:
import numpy as np
import pandas as pd
import spacy
import re
from itertools import chain

In [2]:
dat1 = pd.read_pickle("../data/elmondo_es_qa_no_numbers.pkl")
dat2 = pd.read_pickle("../data/elmondo_es_qa_no_numbers_eco.pkl")
dat3 = pd.read_pickle("../data/elmondo_es_qa_no_numbers_sp.pkl")

In [3]:
dat = pd.concat([dat1, dat2, dat3], axis=0)
dat = dat.reset_index().drop('index', axis=1)

In [4]:
nlp = spacy.load("es")

In [5]:
with open("../wordvecs/wiki.es/wiki.es.nospace.vec") as f:
    f.readline()
    nlp.vocab.load_vectors(f)

In [6]:
dat['question_nlp'] = dat['question'].apply(lambda x: nlp(x, parse=False, tag=False, entity=False))
dat['story_nlp'] = dat['story'].apply(lambda x: nlp(x, parse=False, tag=False, entity=False))

In [7]:
dat.head()

,question,answer,story,question_nlp,story_nlp
0,La información proviene de los documentos del...,@entity318,La @entity57 (@entity187) trabaja en la constr...,"( , La, información, proviene, de, los, docume...","(La, @entity57, (, @entity187, ), trabaja, en,..."
1,Han detenido a 7 personas intregrantes de la ...,@entity299,Esta semana @entity170 ha recibido una noticia...,"( , Han, detenido, a, 7, personas, intregrante...","(Esta, semana, @entity170, ha, recibido, una, ..."
2,@placeholder tiene una gran tasa de criminali...,@entity223,Esta semana @entity223 ha recibido una noticia...,"( , @placeholder, tiene, una, gran, tasa, de, ...","(Esta, semana, @entity223, ha, recibido, una, ..."
3,Los civiles armados se niegan y se registran ...,@entity249,La situación en el @entity163 de @entity179 es...,"( , Los, civiles, armados, se, niegan, y, se, ...","(La, situación, en, el, @entity163, de, @entit..."
4,"Su líder moral, el dr. @placeholder, dice en ...",@entity420,La situación en el @entity47 de @entity355 es ...,"( , Su, líder, moral, ,, el, dr, ., @placehold...","(La, situación, en, el, @entity47, de, @entity..."


In [8]:
dat.shape

(57125, 5)

In [9]:
q_lengths = dat['question_nlp'].apply(len)
s_lengths = dat['story_nlp'].apply(len)

In [10]:
question_has_vec_ind = dat['question_nlp'].apply(lambda x: np.array([w.has_vector for w in x]))

In [11]:
story_has_vec_ind = dat['story_nlp'].apply(lambda x: np.array([w.has_vector for w in x]))

In [12]:
question_ent_ind = dat['question_nlp'].apply(lambda x: np.array([w.text[:7] == '@entity' for w in x]))
story_ent_ind = dat['story_nlp'].apply(lambda x: np.array([w.text[:7] == '@entity' for w in x]))

In [13]:
question_ph_ind = dat['question_nlp'].apply(lambda x: np.array([w.text == '@placeholder' for w in x]))

In [14]:
no_ph_ind = (question_ph_ind.apply(sum) == 0)

In [15]:
no_ans_ind = []
for i in range(dat.shape[0]):
    no_ans_ind.append(re.search(dat['answer'][i], dat['story'][i]) == None)
no_ans_ind = np.array(no_ans_ind)

In [16]:
selected = dat.loc[(q_lengths > 4) & (q_lengths <= 50) & (s_lengths <= 2000) & (~no_ans_ind) & (~(no_ph_ind))]

In [17]:
selected.shape

(56130, 5)

In [18]:
selected.head()

,question,answer,story,question_nlp,story_nlp
0,La información proviene de los documentos del...,@entity318,La @entity57 (@entity187) trabaja en la constr...,"( , La, información, proviene, de, los, docume...","(La, @entity57, (, @entity187, ), trabaja, en,..."
1,Han detenido a 7 personas intregrantes de la ...,@entity299,Esta semana @entity170 ha recibido una noticia...,"( , Han, detenido, a, 7, personas, intregrante...","(Esta, semana, @entity170, ha, recibido, una, ..."
2,@placeholder tiene una gran tasa de criminali...,@entity223,Esta semana @entity223 ha recibido una noticia...,"( , @placeholder, tiene, una, gran, tasa, de, ...","(Esta, semana, @entity223, ha, recibido, una, ..."
3,Los civiles armados se niegan y se registran ...,@entity249,La situación en el @entity163 de @entity179 es...,"( , Los, civiles, armados, se, niegan, y, se, ...","(La, situación, en, el, @entity163, de, @entit..."
4,"Su líder moral, el dr. @placeholder, dice en ...",@entity420,La situación en el @entity47 de @entity355 es ...,"( , Su, líder, moral, ,, el, dr, ., @placehold...","(La, situación, en, el, @entity47, de, @entity..."


In [19]:
sel = selected.drop(['story_nlp', 'question_nlp'], axis=1)

In [20]:
sel.head()

,question,answer,story
0,La información proviene de los documentos del...,@entity318,La @entity57 (@entity187) trabaja en la constr...
1,Han detenido a 7 personas intregrantes de la ...,@entity299,Esta semana @entity170 ha recibido una noticia...
2,@placeholder tiene una gran tasa de criminali...,@entity223,Esta semana @entity223 ha recibido una noticia...
3,Los civiles armados se niegan y se registran ...,@entity249,La situación en el @entity163 de @entity179 es...
4,"Su líder moral, el dr. @placeholder, dice en ...",@entity420,La situación en el @entity47 de @entity355 es ...


In [26]:
re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', '@entity23,@entity67@entity56')

'@entity23, @entity67 @entity56'

In [27]:
deliminated_questions = sel['question'].apply(lambda x: re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', x))
deliminated_stories = sel['story'].apply(lambda x: re.sub(r'(?<=\S)@entity(\d+)', r' @entity\1', x))

In [28]:
sel['question'] = deliminated_questions
sel['story'] = deliminated_stories

In [30]:
dev = sel.sample(frac=0.1, random_state=7777, replace=False)
rest = sel[~np.in1d(sel.index, dev.index)]
test = rest.sample(frac=0.1, random_state=7777, replace=False)
train = rest[~np.in1d(rest.index, test.index)]

In [31]:
print(dev.shape, rest.shape, test.shape, train.shape)

(5613, 3) (50517, 3) (5052, 3) (45465, 3)


In [32]:
train.to_pickle("../input_data/train_es.pkl")
dev.to_pickle("../input_data/dev_es.pkl")
test.to_pickle("../input_data/test_es.pkl")